# 예측 결과 분석 하기

이 노트북은 생성된 결과를 로딩하여 진실값과 비교하여 정확도를 계산하는 노트북 입니다.

---

# 1. 결과 데이터 셋 로딩

In [2]:
dataset_folder = "../data/retail_demo_store/"

import os

## Lexical Result

Lexical_file_path = os.path.join(dataset_folder, "output/aws_opensearch_result01.csv")
Semantic_file_path = os.path.join(dataset_folder, "output/aws_opensearch_result02.csv")
Hybrid_file_path = os.path.join(dataset_folder, "output/aws_opensearch_result03.csv")
reranker_Hybrid_file_path = os.path.join(dataset_folder, "output/aws_opensearch_result04.csv")

import pandas as pd

lexical_df = pd.read_csv(Lexical_file_path)
semantic_df = pd.read_csv(Semantic_file_path)
hybrid_df = pd.read_csv(Hybrid_file_path)
reranker_hybrid_df = pd.read_csv(reranker_Hybrid_file_path)




# 2. 정확도 계산

In [3]:
def show_metrics_from_result(ldf,type, gt_label, pre_label, result_col_name):
    df = ldf.copy()
    
    # Apply function to check if column1 and column2 are equal
    df[result_col_name] = df.apply(lambda row: row[gt_label] == row[pre_label], axis=1)

    num_correct = df[result_col_name].sum()
    # num_incorrect = df[~result_col_name].sum()
    num_total = df[result_col_name].shape[0]
    print(f"Test type: {type}, \n\t accuracy: {round(num_correct / num_total, 3)}, total_sample: {num_total} , correct: {num_correct}, incorrec: {num_total - num_correct}")

    return df

lexical_metric_df = show_metrics_from_result(ldf=lexical_df,
                                    type = "Lexical with Tori",
                                    gt_label='intent', pre_label='result', 
                                    result_col_name = 'correct')

semantic_metric_df = show_metrics_from_result(ldf=semantic_df,
                                    type = "Semantic",
                                    gt_label='intent', pre_label='result', 
                                    result_col_name = 'correct')
hybrid_metric_df = show_metrics_from_result(ldf=hybrid_df,
                                    type = "Hybrid",
                                    gt_label='intent', pre_label='result', 
                                    result_col_name = 'correct')                                    
reranker_hybrid_metric_df = show_metrics_from_result(ldf=reranker_hybrid_df,
                                    type = "Reranker Hybrid",
                                    gt_label='intent', pre_label='result', 
                                    result_col_name = 'correct')                                    



Test type: Lexical with Tori, 
	 accuracy: 0.971, total_sample: 375 , correct: 364, incorrec: 11
Test type: Semantic, 
	 accuracy: 0.965, total_sample: 375 , correct: 362, incorrec: 13
Test type: Hybrid, 
	 accuracy: 0.976, total_sample: 375 , correct: 366, incorrec: 9
Test type: Reranker Hybrid, 
	 accuracy: 0.979, total_sample: 375 , correct: 367, incorrec: 8


# 3. 결과 데이터 셋 보기

In [4]:
reranker_hybrid_metric_df

Unnamed: 0       product                               input      intent  \
0             0         블랙 슈즈                   남성용 블랙 포멀 슈즈 한 켤레       신발|정장   
1             1         스틸 가위                 모든 욕실에 사용할 수 있는 면도기      뷰티|그루밍   
2             2     백합 어레인지먼트    우리 재배자들이 신선하고 활기차게 배달한 백합 어레인지먼트  플로럴|어레인지먼트   
3             3       어쿠스틱 드럼   이 어쿠스틱 드럼은 아마추어 및 전문 뮤지션에게 적합합니다.      악기|타악기   
4             4      어쿠스틱 베이스  이 어쿠스틱 베이스는 아마추어 및 전문 음악가에게 적합합니다.    악기 | 현악기   
..          ...           ...                                 ...         ...   
370         370          돼지고기    우리 돼지고기는 농부들이 지속 가능한 방식으로 사육합니다.      식료품|고기   
371         371     울트라시크 스카프          어떤 상황에도 어울리는 여성용 울트라시크 스카프      의류|스카프   
372         372   고급스러운 세라믹 볼            모든 상황에 어울리는 고급스러운 세라믹 그릇     가정용품|그릇   
373         373    새들 브라운 핸드백       이 새들 브라운 드레시 핸드백은 전형적인 필수품입니다    액세서리|핸드백   
374         374  스틸 블루 스퀘어 쿠션  이 스틸 블루 스퀘어 쿠션은 집에 꼭 있어야 할 필수품입니다.   홈 데코 | 쿠션   

         result  correct  
0         신발|정장     True  
1        뷰티|그루밍     True  
2    플로럴|어레인지먼트     True  
3        악기|타악기     True  
4      악기 | 현악기     True  
..          ...      ...  
370      식료품|고기     True  
371      의류|스카프     True  
372     가정용품|그릇     True  
373    액세서리|핸드백     True  
374   홈 데코 | 쿠션     True  

[375 rows x 6 columns]

위의 테이블을 직관적으로 이해하기 위해서 테이블의 컬럼 이름을 수정 함.

In [13]:
def polish_df(ldf):
    df = ldf.copy()
    df = df.drop("Unnamed: 0", axis=1)
    # rename columns in dataframe
    df.columns = ['product_name', 'product_desc','category_ground_truth',  'category_ground_prediction', 'correct']
    
    return df

reranker_hybrid_metric_polish_df = polish_df(reranker_hybrid_metric_df)
reranker_hybrid_metric_polish_df

product_name                        product_desc category_ground_truth  \
0           블랙 슈즈                   남성용 블랙 포멀 슈즈 한 켤레                 신발|정장   
1           스틸 가위                 모든 욕실에 사용할 수 있는 면도기                뷰티|그루밍   
2       백합 어레인지먼트    우리 재배자들이 신선하고 활기차게 배달한 백합 어레인지먼트            플로럴|어레인지먼트   
3         어쿠스틱 드럼   이 어쿠스틱 드럼은 아마추어 및 전문 뮤지션에게 적합합니다.                악기|타악기   
4        어쿠스틱 베이스  이 어쿠스틱 베이스는 아마추어 및 전문 음악가에게 적합합니다.              악기 | 현악기   
..            ...                                 ...                   ...   
370          돼지고기    우리 돼지고기는 농부들이 지속 가능한 방식으로 사육합니다.                식료품|고기   
371     울트라시크 스카프          어떤 상황에도 어울리는 여성용 울트라시크 스카프                의류|스카프   
372   고급스러운 세라믹 볼            모든 상황에 어울리는 고급스러운 세라믹 그릇               가정용품|그릇   
373    새들 브라운 핸드백       이 새들 브라운 드레시 핸드백은 전형적인 필수품입니다              액세서리|핸드백   
374  스틸 블루 스퀘어 쿠션  이 스틸 블루 스퀘어 쿠션은 집에 꼭 있어야 할 필수품입니다.             홈 데코 | 쿠션   

    category_ground_prediction  correct  
0                        신발|정장     True  
1                       뷰티|그루밍     True  
2                   플로럴|어레인지먼트     True  
3                       악기|타악기     True  
4                     악기 | 현악기     True  
..                         ...      ...  
370                     식료품|고기     True  
371                     의류|스카프     True  
372                    가정용품|그릇     True  
373                   액세서리|핸드백     True  
374                  홈 데코 | 쿠션     True  

[375 rows x 5 columns]